In [1]:
import pandas as pd
import itertools
import pandas as pd
import numpy as np
import random
import math
import os
import socket
import whois
from datetime import datetime
import time
from bs4 import BeautifulSoup
import urllib
import bs4
import os

In [2]:
df = pd.read_csv('Malware_dataset.csv', delimiter=",")

In [3]:
df.duplicated().sum()

8875

In [4]:
df.isnull().sum()

url    0
dtype: int64

In [5]:
# remove duplicated values
df = df.drop_duplicates()

In [3]:
df.shape

(11566, 1)

# Feature Extraction

In [4]:
import re
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4 with port
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '([0-9]+(?:\.[0-9]+){3}:[0-9]+)|'
        '((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

In [5]:
from urllib.parse import urlparse
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

In [6]:
from urllib.parse import urlparse
def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

In [7]:
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

In [8]:
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net', url)
    if match:
        return 1
    else:
        return 0

In [9]:
def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr', url)
    if match:
        return 1
    else:
        return 0

In [10]:
#Importing dependencies
from urllib.parse import urlparse
from tld import get_tld
import os.path

#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

In [11]:
def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1

In [12]:
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

In [13]:
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

In [14]:
# 12.Web traffic (Web_Traffic)
def web_traffic(url):
    try:
    #Filling the whitespaces in the URL if any
        url = urllib.parse.quote(url)
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        rank = int(rank)
    except TypeError:
        return 1
        if rank <100000:
            return 1
        else:
            return 0

In [15]:
df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))
df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))
df['count.'] = df['url'].apply(lambda i: i.count('.'))
df['count-www'] = df['url'].apply(lambda i: i.count('www'))
df['count@'] = df['url'].apply(lambda i: i.count('@'))
df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))
df['count_embed_domian'] = df['url'].apply(lambda i: no_of_embed(i))
df['short_url'] = df['url'].apply(lambda i: shortening_service(i))
df['count-https'] = df['url'].apply(lambda i : i.count('https'))
df['count-http'] = df['url'].apply(lambda i : i.count('http'))
df['count%'] = df['url'].apply(lambda i: i.count('%'))
df['count?'] = df['url'].apply(lambda i: i.count('?'))
df['count-'] = df['url'].apply(lambda i: i.count('-'))
df['count='] = df['url'].apply(lambda i: i.count('='))
#Length of URL
df['url_length'] = df['url'].apply(lambda i: len(str(i)))
#Hostname Length
df['hostname_length'] = df['url'].apply(lambda i: len(urlparse(i).netloc))
df['sus_url'] = df['url'].apply(lambda i: suspicious_words(i))
df['fd_length'] = df['url'].apply(lambda i: fd_length(i))
#Length of Top Level Domain
df['tld'] = df['url'].apply(lambda i: get_tld(i,fail_silently=True))
df['tld_length'] = df['tld'].apply(lambda i: tld_length(i))
df['count-digits']= df['url'].apply(lambda i: digit_count(i))
df['count-letters']= df['url'].apply(lambda i: letter_count(i))
#df['web_traffic']= df['url'].apply(lambda i: web_traffic(i))

In [16]:
df

,url,use_of_ip,abnormal_url,count.,count-www,count@,count_dir,count_embed_domian,short_url,count-https,...,count-,count=,url_length,hostname_length,sus_url,fd_length,tld,tld_length,count-digits,count-letters
0,http://gzzax.livechatvalue.com/chat/chatClient...,0,1,5,0,0,3,0,0,0,...,0,7,378,23,0,4,com,3,106,205
1,http://gzzax.livechatvalue.com/chat/chatClient...,0,1,5,0,0,3,0,0,0,...,0,7,378,23,0,4,com,3,106,205
2,http://gzzax.livechatvalue.com/chat/chatClient...,0,1,5,0,0,3,0,0,0,...,0,7,378,23,0,4,com,3,106,205
3,http://gzzax.livechatvalue.com/chat/chatClient...,0,1,5,0,0,3,0,0,0,...,0,7,378,23,0,4,com,3,106,205
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,0,1,3,0,0,4,0,1,0,...,1,0,268,11,0,11,com.cn,6,83,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11561,http://www.plastischechirurgie.net/gesicht/fac...,0,1,2,1,0,2,0,0,0,...,0,0,54,27,0,7,net,3,0,47
11562,http://www.plastischechirurgie.net/haartranspl...,0,1,2,1,0,1,0,0,0,...,0,0,54,27,0,19,net,3,0,48
11563,http://www.plastischechirurgie.net/korper/gyna...,0,1,2,1,0,2,0,0,0,...,0,0,54,27,0,6,net,3,0,47
11564,http://dl.get1993desk.com/n/50517366/RealPlaye...,0,1,3,0,0,3,0,0,0,...,0,0,53,18,0,1,com,3,12,32


In [17]:
#Extracting the feautres & storing them in a list
malware_features = []
label = 1

for i in range(0, 11566):
    url = df['url'][i]
    malware_features.append(label)

In [18]:
feature_names = ['label']
malware = pd.DataFrame(malware_features, columns= feature_names)
malware.head

<bound method NDFrame.head of        label
0          1
1          1
2          1
3          1
4          1
...      ...
11561      1
11562      1
11563      1
11564      1
11565      1

[11566 rows x 1 columns]>

In [20]:
df['label']= malware

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11566 entries, 0 to 11565
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   url                 11566 non-null  object
 1   use_of_ip           11566 non-null  int64 
 2   abnormal_url        11566 non-null  int64 
 3   count.              11566 non-null  int64 
 4   count-www           11566 non-null  int64 
 5   count@              11566 non-null  int64 
 6   count_dir           11566 non-null  int64 
 7   count_embed_domian  11566 non-null  int64 
 8   short_url           11566 non-null  int64 
 9   count-https         11566 non-null  int64 
 10  count-http          11566 non-null  int64 
 11  count%              11566 non-null  int64 
 12  count?              11566 non-null  int64 
 13  count-              11566 non-null  int64 
 14  count=              11566 non-null  int64 
 15  url_length          11566 non-null  int64 
 16  hostname_length     11

In [23]:
# Storing the data in CSV file
df.to_csv('malware.csv', index=False)